In [1]:
# libraries

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.by import By

import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager as CM
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import time

import pandas as pd
import re
from deep_translator import GoogleTranslator
import tqdm
import os

In [2]:
## Initiating the browser
options = webdriver.ChromeOptions()
path = "../chrome/chromedriver.exe"
service = Service(executable_path=path)
browser = webdriver.Chrome(service=service,options=options)
browser.maximize_window()

browser.set_page_load_timeout(60) # only for 5 minutes


In [26]:
# New way :>
chrome_options = Options()
# Use WebDriverManager to handle driver installation
service = Service(executable_path=CM().install())
browser = webdriver.Chrome(service=service, options=chrome_options)

browser.maximize_window()
browser.set_page_load_timeout(60) # only for 5 minutes

In [17]:
def getEmailFromWebsite(url):
    global browser 
    
    emails = set()
    # opening with selnium
    browser.get(url)
    
    # from any element which has @ in their text
    elems = browser.find_elements("xpath","//*[contains(text(), '@')]")
    for elem in elems:
        es = ((re.findall(r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b", elem.text, re.I)))
        emails.update(es)
    
    if len(emails) == 0:
        try:
            
            time.sleep(1)
            button = browser.find_element(By.XPATH, "//a[contains(text(),'contact')]") 
            browser.execute_script("arguments[0].click();", button)


            elems = browser.find_elements("xpath","//*[contains(text(), '@')]")
            for elem in elems:
                es = ((re.findall(r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b", elem.text, re.I)))
                emails.update(es)
        except:
            pass
    
    return ";".join(emails)


In [4]:
url = "https://smartlog-group.com/"
getEmailFromWebsite(url)

'contact@smartlog-group.com'

In [4]:
def extractWebsiteFromName(name):
    global browser
    global country
    url_base = "https://www.google.com/search?q="
    url = url_base + name + " " + country

    # opening a url:
    browser.get(url)
    elems = browser.find_elements("xpath","//a[@jsname='n1obkb mI8Pwc']")

    # check first
    elem = elems[0]
    link = elem.text.split("\n")[2]
    
    marketplaces = [
        'justdail',
        'amazon',
        'cphi',
        'dnb',
        'wikipedia',
        'linkedin',
        'indiamart'
    ]
    
    for m in marketplaces:
        if m in link:
            return None
    
    if 'http' in link:
        if "›" in link:
            link = link.split("›")[0].strip()

        for m in marketplaces:
            if m in link:
                return None
            
        return link
    else:
        # check second
        elem = elems[1]
        link = elem.text.split("\n")[2]
        if 'http' in link:
            if "›" in link:
                link = link.split("›")[0].strip()
            return link
        else:
            # check third
            elem = elems[1]
            link = elem.text.split("\n")[2]
            if 'http' in link:
                if "›" in link:
                    link = link.split("›")[0].strip()
                return link
    return None



In [36]:
def extractWebsiteGoogleSearch(name):
    global browser
    global country
    url_base = "https://www.google.com/search?q="
    url = url_base + name + " " + country

    # opening a url:
    browser.get(url)
    elems = browser.find_elements("xpath","//a[@class='n1obkb mI8Pwc']")

    # check first
    elem = elems[0]
    link = elem.get_attribute('href')
    
    return link



In [13]:
# loading file of names :> 
df = pd.read_csv("../results/barcelona/Tourism companies.csv")
df = pd.read_csv("../results/industries/Medical shop/Spain/companies_highlevel_1.csv")
df.head()

,name,phone,website,address
0,Comercial Médico Quirúrgica,NaN,NaN,Tienda de material sanitario · · Carrer de Ca...
1,Farmacia Guillermo Chacón Puig,934 35 92 93,http://farmaciaguillermochacon.com/,"Farmacia · · Carrer de la Indústria, 128"
2,Farmàcia Dra. Pascal,934 35 82 68,NaN,"Farmacia · · AL COSTAT HOSP. DE SANT PAU, Av...."
3,Farmacia Gaudi - Maria Belenguer,934 35 80 90,NaN,"Farmacia · · Av. de Gaudí, 71"
4,Ambi Medical,NaN,NaN,Tienda de material sanitario · · Carrer de Sa...


In [24]:
names = list(df['name'].values)
websites = []

In [27]:
country = "spain"
for name in tqdm.tqdm(names[:1]):
    url_base = "https://www.google.com/search?q="
    url = url_base + name + " " + country
    browser.get(url)
    time.sleep(10)
    try:
        # opening a url:
        browser.get(url)
        elems = browser.find_elements("xpath","//a[@class='n1obkb mI8Pwc']")
    
        # check first
        elem = elems[0]
        link = elem.get_attribute('href')
        
        # link = extractWebsiteGoogleSearch(name)
        websites.append(link)
    except:
        pass

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:12<00:00, 12.86s/it]


In [ ]:
country = "spain"
for name in tqdm.tqdm(names[2:]):
    url_base = "https://www.google.com/search?q="
    url = url_base + name + " " + country
    browser.get(url)
    time.sleep(2)
    try:
        # opening a url:
        browser.get(url)
        elems = browser.find_elements("xpath","//a[@class='n1obkb mI8Pwc']")
    
        # check first
        elem = elems[0]
        link = elem.get_attribute('href')
        
        # link = extractWebsiteGoogleSearch(name)
        websites.append(link)
    except:
        pass

 53%|██████████████████████████████████████████▌                                      | 62/118 [04:40<04:02,  4.32s/it]

In [12]:
len(websites)

0

In [54]:
websites_df = pd.DataFrame({"website":websites})
websites_df.to_csv("../results/websites.csv",index=None)

In [10]:
name = "Barcelona da Gema (Turismo em Portugués)"
country = "Barcelona"
links = extractWebsiteFromName(name)
links

'https://www.tripadvisor.es'

In [15]:
elems = browser.find_elements("xpath","//a[@class='n1obkb mI8Pwc']")
elems

[<selenium.webdriver.remote.webelement.WebElement (session="00437ebce947adfd1feb360f207c5549", element="f.06791A7EBA4C2E8BF4E761D57FACE59B.d.869A3B8E29A6D2E98B64AAAB35FC95D4.e.1854")>,
 <selenium.webdriver.remote.webelement.WebElement (session="00437ebce947adfd1feb360f207c5549", element="f.06791A7EBA4C2E8BF4E761D57FACE59B.d.869A3B8E29A6D2E98B64AAAB35FC95D4.e.1870")>]

In [18]:
for e in elems:
    print(e.get_attribute('href'))

http://www.barcelonadagema.com/
https://www.google.com/search?sca_esv=dd2f1766d71666ee&si=APYL9btvhO6SAb8jF9HqTZMMa7vs_teLnZaEVrJZwRKFIIKjocFp6OWbsnWeEeIHEofi20AjgtS5-YhR9Idonf35NQU20Zwu6yy5JbycHi2iv36hfNtSeZvo1oCOpDtgYwF_bABUA4rz9bdI4-lZjeLHHwurQeXe9XPkNo-lEBAMB7zwpDrw-Kk%3D&q=Barcelona+da+Gema+%28Turismo+em+Portugu%C3%A9s%29+Rese%C3%B1as&sa=X&ved=2ahUKEwjT7d7k8PiMAxX6SPEDHf1RFtcQ0bkNegQIIBAE


In [ ]:
websites = [
    'https://weareworldexperience.com/',
    'https://mindtrip.ai',
    'https://www.barcelonadagema.com/'
]

In [6]:
def extractEmail(row):
    
    name = row['name']
    website = row['website']
    
    emails_all = []
    
    if type(website) != float:
        # extract emails from website:
        emails = getEmailFromWebsite(website)
        emails_all.extend(emails.split(";"))
        
    else:
        # try extracting website from company name
        website = extractWebsiteFromName(name)
        row['website']=website
        
        emails = getEmailFromWebsite(website)
        emails_all.extend(emails.split(";"))
    
    row['emails']=";".join(emails_all)
    return row
    


In [57]:
emails = []

In [66]:
websites

['http://weareworldexperience.com/',
 'http://www.barcelonadagema.com/',
 'https://www.barcelonalocalexperiences.com/',
 'https://barcelonadaytrip.com/',
 'https://www.google.com/search?sca_esv=d58a3a4304cf478c&si=APYL9bs7Hg2KMLB-4tSoTdxuOx8BdRvHbByC_AuVpNyh0x2KzZTisYb-5ojl7wl2VGGBoIQHdXdqHZnHvdrc-VODnhfzUjvTwlxhWH7FnVI0kl0tB_qvIOt9R_cLOyp7TPbovtThcUz-&q=Badal+Tours+Rese%C3%B1as&sa=X&ved=2ahUKEwjqyo-_-viMAxUScKQEHUN6DGMQ0bkNegQIKxAD',
 'https://www.inoutbarcelonatours.com/',
 'https://barcelona.city-tour.com/?utm_source=social&utm_medium=Google+My+Business&utm_campaign=GMB_BCT_LinkWeb',
 'https://skipthelinebarcelona.com/',
 'http://www.bcn-adventure.com/',
 'https://explorecatalunya.com/',
 'http://www.belocaltours.com/',
 'https://juliatravel.com/?utm_source=social&utm_medium=Google+My+Business&utm_campaign=GMB_Nord_LinkWeb',
 'http://www.donkeytours.es/',
 'http://www.barcelonartguide.com/',
 'https://www.barcelonaviptour.com/',
 'https://www.touristforum.net/?utm_source=GMB&utm_med

In [70]:
for website in tqdm.tqdm(websites[25+45:]):
    
    email = getEmailFromWebsite(website)
    emails.append(email)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:51<00:00,  2.58s/it]


In [71]:
emails_list = list(set(emails))[1:]

In [72]:
len(emails_list)

29

In [74]:
emails_list_df = pd.DataFrame(emails_list)
emails_list_df.to_csv("../results/emails.csv",index=None)

### Step 2: Selections

In [9]:
country = "India"
industry = "Food"

In [12]:
files = os.listdir("../results/industries/{}/{}/names".format(industry,country))
files

['Bangalore.csv',
 'Chennai.csv',
 'companies_highlevel_1.csv',
 'Delhi.csv',
 'Hyderabad.csv',
 'Kolkata.csv',
 'Mumbai.csv',
 'Pune.csv']

In [ ]:
os.mkdir("../results/industries/{}/{}/emails".format(industry,country))

In [14]:
files[-2:]

['Mumbai.csv', 'Pune.csv']

In [15]:

for ind, file in enumerate(files):
    print("Index ::",ind)
    print("Started for file ::",file)
    
    updated_rows = []
    failed_rows = []
    existing_rows_names = []

    df = pd.read_csv("../results/industries/{}/{}/names/{}".format(industry,country,file))
    rows = df.to_dict('records')
    
    for row in tqdm.tqdm(rows):
        if row['name'] not in existing_rows_names:
            try:
                n = extractEmail(row)
                updated_rows.append(n)
            except:
                failed_rows.append(row)
                row['emails']=""
                updated_rows.append(row)
                pass
            
        existing_rows_names.append(row['name'])
    
    # saving
    df = pd.DataFrame(updated_rows)
    df.to_csv(("../results/industries/{}/{}/emails/{}".format(industry,country,file)),index=None)

Index :: 0
Started for file :: Bangalore.csv


100%|████████████████████████████████████████████████████████████████████████████████| 120/120 [14:11<00:00,  7.09s/it]


Index :: 1
Started for file :: Chennai.csv


100%|████████████████████████████████████████████████████████████████████████████████| 120/120 [15:27<00:00,  7.73s/it]


Index :: 2
Started for file :: companies_highlevel_1.csv


...

100%|████████████████████████████████████████████████████████████████████████████████| 120/120 [12:12<00:00,  6.11s/it]


Index :: 6
Started for file :: Mumbai.csv


100%|████████████████████████████████████████████████████████████████████████████████| 120/120 [10:22<00:00,  5.19s/it]


Index :: 7
Started for file :: Pune.csv


100%|████████████████████████████████████████████████████████████████████████████████| 120/120 [12:01<00:00,  6.01s/it]


In [ ]:
## Ading time break logic with multi-processing:
import multiprocessing
import time

# bar
def bar():
    for i in range(100):
        print "Tick"
        time.sleep(1)

if __name__ == '__main__':
    # Start bar as a process
    p = multiprocessing.Process(target=bar)
    p.start()

    # Wait for 10 seconds or until process finishes
    p.join(10)

    # If thread is still active
    if p.is_alive():
        print "running... let's kill it..."

        # Terminate - may not work if process is stuck for good
        p.terminate()
        # OR Kill - will work for sure, no chance for process to finish nicely however
        # p.kill()

        p.join()

In [22]:
## When it is taking too much time for a certain website:

for ind, file in enumerate(files):
    print("Index ::",ind)
    print("Started for file ::",file)
    
    updated_rows = []
    failed_rows = []
    existing_rows_names = []

    df = pd.read_csv("../results/names/{}/{}".format(country,file))
    rows = df.to_dict('records')
    
    for row in tqdm.tqdm(rows[46:]):
        if row['name'] not in existing_rows_names:
            try:
                n = extractEmail(row)
                updated_rows.append(n)
            except:
                failed_rows.append(row)
                row['emails']=""
                updated_rows.append(row)
                pass
            
        
    
    

Index :: 0
Started for file :: companies_1.csv


100%|██████████████████████████████████████████████████████████████████████████████████| 73/73 [12:20<00:00, 10.15s/it]


In [24]:
# remember to update the index accordingly
updated_rows[0]

{'name': 'Austex Pharmaceutical PTE LTD',
 'phone': '062008 92433',
 'website': 'http://www.austexpharma.com/',
 'address': '81 UBI Avenue 4 #06-03 UB.ONE',
 'emails': 'contact@austexpharma.com'}

In [26]:
# saving
df = pd.DataFrame(updated_rows)
df.to_csv("../results/emails/{}/{}".format(country,file),index=None)